# Dynamic Programming

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import random
from utils import random_graph, create_adjacency_matrix
import math

In [2]:
# If AttributeError: module 'scipy.sparse' has no attribute 'coo_array'
# pip install --upgrade scipy networkx

# Number of nodes in graph
n = 10

# Create a random graph alongside adjacency matrix
graph = random_graph(n)

adj_graph = create_adjacency_matrix(graph)
dist = adj_graph.todense().tolist()
print(dist)

[[0.0, 0.2120870340493113, 0.6576682794610186, 0.9244080444119331, 0.12853301132248018, 0.4443550309435013, 0.6092958826559515, 0.7431317214443087, 0.4293598295670896, 0.4480115970140055], [0.2120870340493113, 0.0, 0.7065660886252252, 1.0864925318761218, 0.08671300752384518, 0.6562980294256855, 0.7013540548247943, 0.8585541374022173, 0.22946887665963753, 0.6516458235380864], [0.6576682794610186, 0.7065660886252252, 0.0, 0.6112312741663286, 0.6621695792020695, 0.7814069168624441, 0.14668402871375819, 0.2787598704387091, 0.7230889685906086, 0.8875616184380217], [0.9244080444119331, 1.0864925318761218, 0.6112312741663286, 0.0, 1.0081734529963984, 0.6997078273697546, 0.4879380474695943, 0.3332339687989218, 1.2067364948752664, 0.8311795318010602], [0.12853301132248018, 0.08671300752384518, 0.6621695792020695, 1.0081734529963984, 0.0, 0.5724308500979127, 0.6420177560553257, 0.7929187962750398, 0.3018285666591801, 0.5744960268953639], [0.4443550309435013, 0.6562980294256855, 0.781406916862444

/Users/danielpark/Documents/directed-salesman/utils.py:123: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  return nx.adjacency_matrix(graph)


In [3]:
def dp_runner(i, bitmask):
    """
    Traverses the graph and finds the 
    
    :graph: graph represented as a adjacency matrix
    :i: ith node
    :bitmask: represents the remaining nodes in the subset (TODO: bits are faster to operate)
    """
    
    # Memoize for Top-Down Recursion
    memo = [[-1]*(1 << (n)) for _ in range(n)]
    
    # Base case:
    # - if only ith bit and 1st bit is set in our bitmask,
    # - it implies we have visited all other nodes already
    if bitmask == ((1 << i) | 3):
        return dist[1][i]
  
    # memoization to store visited distances
    if memo[i][bitmask] != -1:
        return memo[i][bitmask]
  
    res = 10**9  # result of this sub-problem
  
    # Travel to all nodes j in mask and end the path at ith node
    for j in range(1, n):
        if (bitmask & (1 << j)) != 0 and j != i and j != 1:
            res = min(res, dp_runner(j, bitmask & (~(1 << i))) + dist[j][i])

    memo[i][bitmask] = res  # storing the minimum value
    return res

In [4]:
# Driver program
cost = 10**9
for i in range(1, n):
    # try to go from node 1 visiting all nodes in between to i
    # then return from i taking the shortest route to 1
    cost = min(cost, dp_runner(i, (1 << n)-1) + dist[i][1])

print("The cost of most efficient path = " + str(cost))

The cost of most efficient path = 3.0989468965126545


## Sources

1. https://www.youtube.com/watch?v=Q4zHb-Swzro&t=118s&ab_channel=AbdulBari
2. http://www.lsi.upc.edu/~mjserna/docencia/algofib/P07/dynprog.pdf
3. https://medium.com/basecs/speeding-up-the-traveling-salesman-using-dynamic-programming-b76d7552e8dd
4. https://networkx.org/documentation/stable/reference/generated/networkx.linalg.graphmatrix.adjacency_matrix.html
5. https://people.eecs.berkeley.edu/~vazirani/algorithms/chap6.pdf